In [1]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages',
                      '../src']
sys.prefix = '/home/joaom/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml
import matplotlib.pyplot as plt 
import datetime

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import yaml
from jinja2 import Template
cf.themes.THEMES['custom'] = yaml.load(open('cufflinks_template.yaml', 'r'))

In [2]:
from src import utils
from datetime import datetime
conn = utils.connect_athena(path='../configs/athena.yaml')

In [6]:
df = pd.read_sql_query('''
select region_slug, month, day, sum_length
from spd_sdv_waze_corona.prod_daily_daily
where region_slug in (
		select region_slug
		from spd_sdv_waze_corona.prod_analysis_metadata_variation
		where country_iso = 'BR'
		and region_type = 'state')
and month = 6
order by 1, 2, 3
''', conn)

In [ ]:
df['date'] = df.apply(lambda row: pd.Timestamp(year=2020, month=int(row['month']), day=int(row['day'])), 1)

In [20]:
df.pivot(columns='region_slug', index='day', values='sum_length').iplot(
    theme='custom',
    xTitle='Day (June)',
    yTitle='TCI',
    title='TCI for Brazilian States in June 2020',
    legend=False
)

In [15]:
d = df.pivot(columns='region_slug', index='day', values='sum_length')

In [51]:
df = pd.read_sql_query('''
select region_slug, month, day, sum_length
from spd_sdv_waze_corona.prod_daily_daily
where region_slug in (
		select region_slug
		from spd_sdv_waze_corona.prod_analysis_metadata_variation
		where country_iso = 'BR'
		and region_type = 'city')
and month in (5, 6)
order by 1, 2, 3
''', conn)

In [52]:
df['date'] = df.apply(lambda row: pd.Timestamp(year=2020, month=int(row['month']), day=int(row['day'])), 1)

In [55]:
df.pivot(columns='region_slug', index='date', values='sum_length').iplot(
    theme='custom',
    xTitle='Day (June)',
    yTitle='TCI',
    title='TCI for Brazilian States in June 2020',
    legend=False,
    vline=pd.Timestamp(year=2020, month=6, day=10)
)

In [60]:
d = df.pivot(columns='region_slug', index='date', values='sum_length')

In [62]:
d.divide(d.iloc[0]).iplot(
    theme='custom',
    xTitle='Day (June)',
    yTitle='Normalized TCI',
    title='Normalized TCI for Brazilian States in June 2020',
#     legend=False,
    vline=pd.Timestamp(year=2020, month=6, day=10)
)

In [19]:
d.divide(d.iloc[0]).iplot(
    theme='custom',
    xTitle='Day (June)',
    yTitle='Normalized TCI',
    title='Normalized TCI for Brazilian States in June 2020',
    legend=False
)

In [25]:
city =  pd.read_sql_query('''
select 
	s.month,
	s.day,
	br.sum_length as br_sum_length,
	state_sum city_sum,
	state_sum / br.sum_length city_frac
from (
	select  month, day, sum_length
	from spd_sdv_waze_corona.prod_daily_daily
	where region_slug = 'country_brazil'
	order by month, day
) br
join (
	select month, day, sum(sum_length) state_sum
	from spd_sdv_waze_corona.prod_daily_daily
	where region_slug in (
		select region_slug
		from spd_sdv_waze_corona.prod_analysis_metadata_variation
		where country_iso = 'BR'
		and region_type = 'city')
	group by month, day) s
on s.month = br.month
and s.day = br.day
order by 1, 2
''', conn)

In [28]:
city['date'] = city.apply(lambda row: pd.Timestamp(year=2020, month=int(row['month']), day=int(row['day'])), 1)

In [35]:
city.set_index('date')['city_frac'].iplot(
    theme='custom',
    title='City TCI / Country TCI for Brazil',
    yTitle= 'City TCI / Country TCI',
    vline=pd.Timestamp(year=2020, month=6, day=10)
)

In [36]:
city =  pd.read_sql_query('''
select 
	s.month,
	s.day,
	br.sum_length as br_sum_length,
	state_sum city_sum,
	state_sum / br.sum_length city_frac
from (
	select  month, day, sum_length
	from spd_sdv_waze_corona.prod_daily_daily
	where region_slug = 'country_brazil'
	order by month, day
) br
join (
	select month, day, sum(sum_length) state_sum
	from spd_sdv_waze_corona.prod_daily_daily
	where region_slug in (
		select region_slug
		from spd_sdv_waze_corona.prod_analysis_metadata_variation
		where country_iso = 'BR'
		and region_type = 'state')
	group by month, day) s
on s.month = br.month
and s.day = br.day
order by 1, 2
''', conn)

In [37]:
city['date'] = city.apply(lambda row: pd.Timestamp(year=2020, month=int(row['month']), day=int(row['day'])), 1)

In [39]:
city.set_index('date')['city_frac'].iplot(
    theme='custom',
    title='State TCI / Country TCI for Brazil',
    yTitle= 'State TCI / Country TCI',
    vline=pd.Timestamp(year=2020, month=6, day=10)
)

# Brazil Counts

In [95]:
unique = pd.read_sql_query('''select month, day, count(*) unique_uuids
from (
	select distinct uuid, month, day
	from "p-waze-parquet-waze".jams
	where regexp_like(datetime, '202005|202006')
	and country = 'BR')
group by month, day
order by day''', conn)

In [96]:
records = pd.read_sql_query('''select month, day, count(*) n_records, sum(length) sum_length
from "p-waze-parquet-waze".jams
where regexp_like(datetime, '202005|202006')
and country = 'BR'
group by month, day''', conn)

In [97]:
unique['date'] = unique.apply(lambda row: pd.Timestamp(year=2020, month=int(row['month']), day=int(row['day'])), 1)
records['date'] = records.apply(lambda row: pd.Timestamp(year=2020, month=int(row['month']), day=int(row['day'])), 1)

In [98]:
d = (records.set_index('date')
     .sort_index()[['n_records', 'sum_length']]
     .merge(unique.set_index('date')[['unique_uuids']], right_index=True, left_index=True))
d.divide(d.iloc[0]).iplot(
    theme='custom',
    title='Variation of Metrics for Brazil',
    yTitle='Normalized Variation',
    vline=pd.Timestamp(year=2020, month=6, day=10)
)

# Rest of LAC Counts

In [89]:
unique = pd.read_sql_query('''select month, day, count(*) unique_uuids
from (
	select distinct uuid, month, day
	from "p-waze-parquet-waze".jams
	where regexp_like(datetime, '202005|202006')
	and country != 'BR')
group by month, day
order by day''', conn)

In [90]:
records = pd.read_sql_query('''select month, day, count(*) n_records, sum(length) sum_length
from "p-waze-parquet-waze".jams
where regexp_like(datetime, '202005|202006')
and country != 'BR'
group by month, day''', conn)

In [91]:
unique['date'] = unique.apply(lambda row: pd.Timestamp(year=2020, month=int(row['month']), day=int(row['day'])), 1)
records['date'] = records.apply(lambda row: pd.Timestamp(year=2020, month=int(row['month']), day=int(row['day'])), 1)

In [94]:
d = (records.set_index('date')
     .sort_index()[['n_records', 'sum_length']]
     .merge(unique.set_index('date')[['unique_uuids']], right_index=True, left_index=True))
d.divide(d.iloc[0]).iplot(
    theme='custom',
    title='Variation of Metrics for LA except Brazil',
    yTitle='Normalized Variation fixed in May 1',
    vline=pd.Timestamp(year=2020, month=6, day=10)
)

# Per Level

In [105]:
records = pd.read_sql_query('''
select month, day, level, count(*) n_records, sum(length) sum_length
from "p-waze-parquet-waze".jams
where regexp_like(datetime, '202002|202003|202004|202005|202006')
and country = 'BR'
group by month, day, level
''', conn)

In [106]:
records['date'] = records.apply(lambda row: pd.Timestamp(year=2020, month=int(row['month']), day=int(row['day'])), 1)

In [107]:
records.pivot(index='date', columns='level', values='n_records').iplot(
    theme='custom',
    title='Number of Records per Level for Brazil',
    yTitle='Number of Records',
    vline=pd.Timestamp(year=2020, month=6, day=10)
)

In [110]:
records.pivot(index='date', columns='level', values='sum_length').iplot(
    theme='custom',
    title='Total Length per Level for Brazil',
    yTitle='Total Length',
    vline=pd.Timestamp(year=2020, month=6, day=10)
)

In [108]:
records.head()

,month,day,level,n_records,sum_length,date
0,5,18,4,257984,98751417,2020-05-18
1,3,9,1,180600,521851253,2020-03-09
2,3,12,3,2720538,1580328730,2020-03-12
3,3,25,2,293336,382072892,2020-03-25
4,6,3,5,2641789,787894164,2020-06-03


# Per Hour

In [ ]:
df = pd.read_sql_query('''
select region_slug, month, day, sum_length
from spd_sdv_waze_corona.prod_daily_daily
where region_slug in (
		select region_slug
		from spd_sdv_waze_corona.prod_analysis_metadata_variation
		where country_iso = 'BR'
		and region_type = 'city')
and month in (5, 6)
order by 1, 2, 3
''', conn)

In [125]:
res = pd.read_sql_query('''
select year, month, day, hour, region_slug, sum(tci) tci
from spd_sdv_waze_corona.dev_daily_daily
where month = 6
and day in (1,2,3,4,5,6,7,8, 9, 10, 11, 12)
and region_slug = 'saopaulo'
group by year, month, day, hour, region_slug
''', conn)

In [126]:
res['date'] = res.apply(lambda row: pd.Timestamp(year=2020, month=int(row['month']), 
                                                 day=int(row['day']), hour=row['hour']), 1)

In [127]:
res.set_index('date').sort_index()['tci'].iplot(
    theme='custom',
    title='TCI for São Paulo',
    yTitle='TCI',
    vline=pd.Timestamp(year=2020, month=6, day=10)
)